<a href="https://colab.research.google.com/github/WittmannF/machine-learning-web-app/blob/master/3-machine-learning/3_machine_learning_brainstorming_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
Plano
- Importar dados
- Definir Conj. Validação
- Definir Benchmark
- Comparar vários regressores
    - LightGBM
    - XGBoost
- Selecionar top 3 para calibração
- Validação Avançada (???? mudar para stacking.. mudar para blending)
"""


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [0]:
URL_TREINO = 'https://raw.githubusercontent.com/WittmannF/machine-learning-web-app/master/2-preprocessamento/treino_preprocessado.csv'
URL_TESTE = 'https://raw.githubusercontent.com/WittmannF/machine-learning-web-app/master/2-preprocessamento/teste_preprocessado.csv'

In [0]:
treino = pd.read_csv(URL_TREINO)

In [0]:
treino.head()

,norte,oeste,sul,quartos,area,preco
0,1,0,0,1.386294,5.707110,7.378384
1,0,0,1,1.386294,5.017280,8.294300
2,1,0,0,1.609438,5.993961,8.699681
3,0,0,0,1.098612,4.025352,6.857514
4,0,0,1,1.609438,6.206576,8.517393


In [0]:
teste = pd.read_csv(URL_TESTE); teste.head()

,norte,oeste,sul,quartos,area,preco
0,0,0,1,1.098612,5.017280,8.294300
1,0,0,1,1.098612,4.795791,7.438972
2,0,1,0,1.386294,5.860786,9.392745
3,0,0,1,1.609438,5.831882,9.433564
4,0,0,0,1.386294,5.525453,7.170888


In [0]:

X_train, X_val, y_train, y_val = train_test_split(treino.drop('preco', axis=1), 
                                                  treino['preco'], test_size=0.2,
                                                  random_state=5)

In [0]:
X_test = teste.drop('preco', axis=1)

In [0]:
y_test = teste['preco']

In [0]:
X_train.describe()

,norte,oeste,sul,quartos,area
count,687.000000,687.000000,687.000000,687.000000,687.000000
mean,0.244541,0.253275,0.254731,1.212308,4.798655
std,0.430128,0.435204,0.436027,0.361700,0.914110
min,0.000000,0.000000,0.000000,0.693147,0.693147
25%,0.000000,0.000000,0.000000,1.098612,4.110874
50%,0.000000,0.000000,0.000000,1.098612,4.836282
75%,0.000000,1.000000,1.000000,1.386294,5.467918
max,1.000000,1.000000,1.000000,2.890372,6.908755


In [0]:
X_val.describe()

,norte,oeste,sul,quartos,area
count,172.000000,172.000000,172.000000,172.000000,172.000000
mean,0.279070,0.238372,0.220930,1.218667,4.844481
std,0.449852,0.427332,0.416085,0.382570,0.852272
min,0.000000,0.000000,0.000000,0.693147,2.397895
25%,0.000000,0.000000,0.000000,1.098612,4.244424
50%,0.000000,0.000000,0.000000,1.242453,4.795791
75%,1.000000,0.000000,0.000000,1.386294,5.442418
max,1.000000,1.000000,1.000000,2.639057,6.685861


In [0]:
X_test.head()

,norte,oeste,sul,quartos,area
0,0,0,1,1.098612,5.017280
1,0,0,1,1.098612,4.795791
2,0,1,0,1.386294,5.860786
3,0,0,1,1.609438,5.831882
4,0,0,0,1.386294,5.525453


In [0]:
y_test.head()

0    8.294300
1    7.438972
2    9.392745
3    9.433564
4    7.170888
Name: preco, dtype: float64

## Benchmarks
- Linear Regression
    - Train: 0.75
    - Valid: 0.74
    - Test: 0.69

- Random Forest Regression
    - Train: 0.92
    - Valid: 0.71
    - Test: 0.68

- Dummy Regressor
    - Train: 0
    - Valid: 0
    - Test: 0
- Comparação de R2: https://gist.github.com/WittmannF/02060b45ce3ec9239898a5b91df2564e

### Regressao Linear

In [0]:
reg = LinearRegression()

In [0]:
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
reg.coef_

array([-0.13800536,  0.28200382,  0.34185494,  0.76209028,  0.67239302])

In [0]:
np.expm1(reg.predict([[0, 1, 0, np.log1p(2), np.log1p(80)]]))

array([2231.86086135])

In [0]:
y_pred = reg.predict(X_val)

In [0]:
r2_score(y_val, y_pred)

0.7406884642807015

In [0]:
reg.score(X_val, y_val)

0.7406884642807015

In [0]:
reg.score(X_test, y_test)

0.6934300416682773

In [0]:
reg.score(X_train, y_train)

0.7496802277459145

### Random Forest

In [0]:
reg = RandomForestRegressor()

In [0]:
reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
reg.score(X_val, y_val)

0.7148647546603686

In [0]:
reg.score(X_train, y_train)

0.9240504087434748

In [0]:
reg.score(X_test, y_test)

0.6894042614596895

### Dummy Regression

In [0]:
reg = DummyRegressor(strategy="median")

In [0]:
reg.fit(X_train, y_train)

DummyRegressor(constant=None, quantile=None, strategy='median')

In [0]:
reg.score(X_train, y_train)

-0.007027075603127208

In [0]:
reg.score(X_val, y_val)

-0.001471766822955134

In [0]:
reg.score(X_test, y_test)

-0.0036524107803874717

## Comparar Vários Estimadores

In [0]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoLars, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor 

In [0]:
reg_list = [Ridge(),
            Lasso(),
            ElasticNet(),
            LassoLars(),
            HuberRegressor(),
            LGBMRegressor(), 
            XGBRegressor(objective='reg:squarederror'),
            SVR(),
            KNeighborsRegressor(),
            AdaBoostRegressor(),
            GradientBoostingRegressor(),
            MLPRegressor()
            ]

In [0]:
for reg in reg_list:
    print(f'Treinando Modelo {reg.__class__.__name__}')
    reg.fit(X_train, y_train)
    valid_score = reg.score(X_val, y_val)
    train_score = reg.score(X_train, y_train)
    print(f"R2 Score Train: {train_score}")
    print(f"R2 Score Validation: {valid_score}")
    print('='*80)

Treinando Modelo Ridge
R2 Score Train: 0.7496665832422033
R2 Score Validation: 0.7407889913858767
Treinando Modelo Lasso
R2 Score Train: 0.0
R2 Score Validation: -0.0027953964166256284
Treinando Modelo ElasticNet
R2 Score Train: 0.27829550432043415
R2 Score Validation: 0.2733651610565253
Treinando Modelo LassoLars
R2 Score Train: 0.0
R2 Score Validation: -0.0027953964166256284
Treinando Modelo HuberRegressor
R2 Score Train: 0.7465383530063975
R2 Score Validation: 0.7443777999673871
Treinando Modelo LGBMRegressor
R2 Score Train: 0.8600265993797606
R2 Score Validation: 0.7296808852922779
Treinando Modelo XGBRegressor
R2 Score Train: 0.8537212488769671
R2 Score Validation: 0.7146960628726093
Treinando Modelo SVR
R2 Score Train: 0.7982029465453218
R2 Score Validation: 0.7674681850215783
Treinando Modelo KNeighborsRegressor
R2 Score Train: 0.843330694852111
R2 Score Validation: 0.7395389869607639
Treinando Modelo AdaBoostRegressor
R2 Score Train: 0.7921624700562627
R2 Score Validation: 0.71

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
reg_list = [LGBMRegressor(),
            SVR(),
            MLPRegressor()
            ]

In [0]:
for reg in reg_list:
    print(f'Treinando Modelo {reg.__class__.__name__}')
    reg.fit(X_train, y_train)
    valid_score = reg.score(X_val, y_val)
    train_score = reg.score(X_train, y_train)
    test_score = reg.score(X_test, y_test)
    print(f"R2 Score Train: {train_score}")
    print(f"R2 Score Validation: {valid_score}")
    print(f"R2 Score Test: {test_score}")
    print('='*80)

Treinando Modelo LGBMRegressor
R2 Score Train: 0.8600265993797606
R2 Score Validation: 0.7296808852922779
R2 Score Test: 0.7274383812166116
Treinando Modelo SVR
R2 Score Train: 0.7982029465453218
R2 Score Validation: 0.7674681850215783
R2 Score Test: 0.7383375935607613
Treinando Modelo MLPRegressor
R2 Score Train: 0.7879094911148011
R2 Score Validation: 0.7541682566430974
R2 Score Test: 0.7058543538777314


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### GridSearchCV

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
X = treino.drop('preco', axis=1)
y = treino['preco']

In [0]:
param = {'kernel' : ('linear', 'poly', 'rbf'),
         'C' : [1,5,10]},

modelsvr = SVR()

grids = GridSearchCV(modelsvr,param,cv=5)

grids.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=({'C': [1, 5, 10],
                          'kernel': ('linear', 'poly', 'rbf')},),
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
grids.best_params_

{'C': 5, 'kernel': 'rbf'}

In [0]:
SVR()

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
grids.score(X_test, y_test)

0.7460944176955905

#### LightGBM

In [0]:
gridParams = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40, 100],
    'num_leaves': [6, 16, 31],
    #'boosting_type' : ['gbdt'],
    #'objective' : ['binary'],
    #'random_state' : [501], # Updated from 'seed'
    #'colsample_bytree' : [0.65, 0.66],
    #'subsample' : [0.7,0.75],
    #'reg_alpha' : [1,1.2],
    #'reg_lambda' : [1,1.2,1.4],
    }

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

In [0]:
estimator = LGBMRegressor()

In [0]:
gbm = GridSearchCV(estimator, gridParams, cv=5)
gbm.fit(X, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'n_estimators': [20, 40, 100],
                         'num_leaves': [6, 16, 31]},
             

In [0]:
gbm.best_params_

{'learning_rate': 0.1, 'n_estimators': 40, 'num_leaves': 16}

In [0]:
LGBMRegressor()

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
gbm.score(X_test, y_test)

0.7419033049737831

### MLP

In [0]:
param = {
         'learning_rate_init' : [0.0001,0.001,0.01],
         'hidden_layer_sizes':[(50,), (100,), (200,)]
         }

mlp = MLPRegressor()

gmlp = GridSearchCV(mlp,param,cv=5)

gmlp.fit(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=200, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_state=None, shuffle=True,
                                    solver='adam', tol=0.0001,
                                    validation_fraction=0.1, verbose=False,
                                    warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'hidden_laye

In [0]:
gmlp.best_params_

{'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01}

In [0]:
gmlp.score(X_test, y_test)

0.7195084214300906

### Stacking

In [0]:
from sklearn.ensemble import StackingRegressor

In [0]:
estimators = [
    ('svr', SVR(C=5.0,
               kernel='rbf')
    ),
    ('lgb', LGBMRegressor(learning_rate=0.1,
                          n_estimators=40,
                          num_leaves=16)
    
    ),
    ('mlp',MLPRegressor(hidden_layer_sizes=(200,),
                        learning_rate='adaptive',
                        learning_rate_init=0.01
                        )
    )
]
reg = StackingRegressor(
    estimators=estimators
)

reg.fit(X, y)


StackingRegressor(cv=None,
                  estimators=[('svr',
                               SVR(C=5.0, cache_size=200, coef0=0.0, degree=3,
                                   epsilon=0.1, gamma='scale', kernel='rbf',
                                   max_iter=-1, shrinking=True, tol=0.001,
                                   verbose=False)),
                              ('lgb',
                               LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_we...
                                            epsilon=1e-08,
                                            hidden_layer_sizes=(200,),
          

In [0]:
estimators = [
    ('svr', SVR()
    ),
    ('lgb', LGBMRegressor()
    
    ),
    ('mlp',MLPRegressor()
    )
]
reg = StackingRegressor(
    estimators=estimators
)

reg.fit(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


StackingRegressor(cv=None,
                  estimators=[('svr',
                               SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                   epsilon=0.1, gamma='scale', kernel='rbf',
                                   max_iter=-1, shrinking=True, tol=0.001,
                                   verbose=False)),
                              ('lgb',
                               LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_we...
                                            epsilon=1e-08,
                                            hidden_layer_sizes=(100,),
          

In [0]:
reg.score(X_test, y_test)

0.7442672301507383

### Blending

In [0]:
regs = [grids, gbm, gmlp] #SVR, LGB, MLP

all_preds = []
for reg in regs:
    all_preds.append(reg.predict(X_test))

In [0]:
y_pred = np.median(all_preds, axis=0)

In [0]:
r2_score(y_test, y_pred)

0.7459546577138984

In [0]:
y_pred = np.mean(all_preds, axis=0)
r2_score(y_test, y_pred)

0.7431020631456275

### Advanced Validation

In [0]:
reg_list = [Ridge(),
            Lasso(),
            ElasticNet(),
            LassoLars(),
            HuberRegressor(),
            LGBMRegressor(), 
            XGBRegressor(objective='reg:squarederror'),
            SVR(),
            KNeighborsRegressor(),
            AdaBoostRegressor(),
            GradientBoostingRegressor(),
            MLPRegressor()
            ]

In [0]:
from sklearn.model_selection import cross_val_score, cross_val_predict
import warnings
warnings.filterwarnings("ignore", message="ConvergenceWarning")

In [0]:
for reg in reg_list:
    print(f'Treinando Modelo {reg.__class__.__name__}')
    cv_score = cross_val_score(reg, X, y)
    #y_pred = cross_val_predict(reg, X_test, y_test)
    print(f"Cross Validated R2 Score: {np.mean(cv_score):.3f} +/- {np.std(cv_score):.3f}")
    #print(f"Test Score: {r2_score(y_test, y_pred)}")
    print('='*80)

Treinando Modelo Ridge
Cross Validated R2 Score: 0.742 +/- 0.028
Treinando Modelo Lasso
Cross Validated R2 Score: -0.020 +/- 0.022
Treinando Modelo ElasticNet
Cross Validated R2 Score: 0.246 +/- 0.021
Treinando Modelo LassoLars
Cross Validated R2 Score: -0.020 +/- 0.022
Treinando Modelo HuberRegressor
Cross Validated R2 Score: 0.739 +/- 0.026
Treinando Modelo LGBMRegressor
Cross Validated R2 Score: 0.773 +/- 0.033
Treinando Modelo XGBRegressor
Cross Validated R2 Score: 0.774 +/- 0.040
Treinando Modelo SVR
Cross Validated R2 Score: 0.785 +/- 0.026
Treinando Modelo KNeighborsRegressor
Cross Validated R2 Score: 0.752 +/- 0.033
Treinando Modelo AdaBoostRegressor
Cross Validated R2 Score: 0.748 +/- 0.033
Treinando Modelo GradientBoostingRegressor
Cross Validated R2 Score: 0.766 +/- 0.040
Treinando Modelo MLPRegressor


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Cross Validated R2 Score: 0.764 +/- 0.026


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### Nested Cross Validation

In [0]:
regs = [grids, gbm, gmlp] #SVR, LGB, MLP

for reg in regs:
    print(f'Treinando Modelo {reg.__class__.__name__}')
    scores = cross_val_score(reg, X, y)
    print(f"Cross Validated R2 Score: {np.mean(scores):.3f} +/- {np.std(scores):.3f}")
    print('='*80)

Treinando Modelo GridSearchCV
Cross Validated R2 Score: 0.786 +/- 0.027
Treinando Modelo GridSearchCV
Cross Validated R2 Score: 0.774 +/- 0.032
Treinando Modelo GridSearchCV


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Cross Validated R2 Score: 0.783 +/- 0.024
